# SQL 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Задания

- 1) Посчитайте, сколько книг вышло после 1 января 2000 года;
- 2) Для каждой книги посчитайте количество обзоров и среднюю оценку;
- 3) Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- 4) Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- 5) Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Подготовка к работе

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [2]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Задание 1

In [3]:
task = '''
    SELECT COUNT(book_id),
           COUNT(title)
    FROM books
    WHERE publication_date > '1999-12-31'
'''

In [4]:
pd.io.sql.read_sql(task, con = engine) 

,count,count
0,821,821


**C начала 2000 года была выпущена 821 книга.
Дополнильно проверили по названию книг.** 

### Задание 2

In [5]:
task_2 = '''
          SELECT book_id,
                 COUNT(review_id) AS count_review,
                 ROUND(AVG(rating), 2) AS avg_rating
          FROM (SELECT re.book_id,
                       re.review_id,
                       rat.rating
                FROM reviews AS re
                INNER JOIN ratings AS rat ON re.book_id = rat.book_id) AS filt   
          GROUP BY book_id
          ORDER BY avg_rating DESC
'''

In [6]:
#pd.io.sql.read_sql(task_2, con = engine)

In [7]:
task_2 = '''
SELECT b.title,
       b.book_id,
    rev.review_count AS review_count,
    rat.rating_avg AS rating_avg
FROM books AS b 
LEFT JOIN (SELECT book_id,
                  AVG(rating) AS rating_avg
           FROM ratings
           GROUP BY book_id) AS rat ON rat.book_id = b.book_id
LEFT JOIN (SELECT book_id,
                  COUNT(review_id) AS review_count
          FROM reviews
          GROUP BY book_id) AS rev ON rev.book_id = b.book_id
ORDER BY review_count DESC;
'''              

In [8]:
pd.io.sql.read_sql(task_2, con = engine)

,title,book_id,review_count,rating_avg
0,The Natural Way to Draw,808,NaN,3.000000
1,Disney's Beauty and the Beast (A Little Golden...,191,NaN,4.000000
2,Essential Tales and Poems,221,NaN,4.000000
3,Anne Rice's The Vampire Lestat: A Graphic Novel,83,NaN,3.666667
4,Leonardo's Notebooks,387,NaN,4.000000
...,...,...,...,...
995,Stone of Farewell (Memory Sorrow and Thorn #2),599,1.0,5.000000
996,Debt of Honor (Jack Ryan #7),187,1.0,3.000000
997,Color: A Natural History of the Palette,156,1.0,4.333333
998,Winter Prey (Lucas Davenport #5),984,1.0,4.500000


**Количество обзоров и средняя оценка книг**

### Задание 3

In [9]:
task_3 = """
         SELECT publisher,
                COUNT(book_id) 
         FROM publishers AS pub
         INNER JOIN books as bo ON pub.publisher_id = bo.publisher_id
         WHERE num_pages > 50
         GROUP BY publisher
         ORDER BY COUNT DESC
         LIMIT 1
"""

In [10]:
pd.io.sql.read_sql(task_3, con = engine)

,publisher,count
0,Penguin Books,42


**Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц**

### Задание 4

In [11]:
task_4 = """
       SELECT a.author,
       ROUND(AVG(rat.rating), 2) as rating_avg
FROM authors AS a
LEFT JOIN books AS b ON b.author_id=a.author_id
LEFT JOIN ratings AS rat ON rat.book_id=b.book_id
WHERE b.book_id IN (SELECT  book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating) >= 50)
GROUP BY a.author
ORDER BY AVG(rat.rating) DESC
LIMIT 1
"""

In [12]:
pd.io.sql.read_sql(task_4, con = engine)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


### Задание 5

In [14]:
task_5 = """
WITH
reviews AS (SELECT username,
                   COUNT(review_id) AS count_review
            FROM reviews
            GROUP BY username),
            
ratings AS (SELECT username,
                   COUNT(rating_id) AS count_rating
            FROM ratings 
            GROUP BY username)
            
SELECT ROUND(AVG(reviews.count_review), 1)
FROM reviews JOIN ratings ON reviews.username = ratings.username
WHERE ratings.count_rating > 50
"""

In [15]:
pd.io.sql.read_sql(task_5, con = engine)

,round
0,24.3


**24.33 - среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**